In [ ]:
#there's an "MNE-BIDS" package for downloading and working straight from open_neuro
# https://mne.tools/mne-bids/dev/auto_examples/read_bids_datasets.html

# .. note: If the keyword argument include is left out of
#          ``openneuro.download``, the whole dataset will be downloaded.
#          We're just using data from one subject to reduce the time
#          it takes to run the example.
import os
import os.path as op
import openneuro
#to prevent runtime error
import nest_asyncio
# nest_asyncio.apply()
#__import__('IPython').embed()

from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report

dataset = 'ds003478' #already changed to Cavanagh's
subject = '001'
print(sample.data_path())

# Download one subject's data from each dataset
bids_root = op.join(op.dirname('/Users/melodyyang/PycharmProjects/COG403/venv/Subjects'), dataset)
if not op.isdir(bids_root):
    os.makedirs(bids_root)

openneuro.download(dataset=dataset, target_dir=bids_root)

In [ ]:
print_dir_tree(bids_root)
# print(make_report(bids_root))

In [ ]:
#separating the runs -- each run has same parameters, maybe just diff order of tasks but this is just resting state so doesn't matter too much
# bids_path = BIDSPath(root=bids_root, run=run, datatype=datatype)
# print(bids_path.match())

In [ ]:
#separating tasks to read data -- all in this database are rest tasks (O- Open eyes; C - Closed eyes)
datatype = 'eeg'
run = '02'
task = 'Rest'
suffix = 'eeg'
subject = '001'
from mne_bids import BIDSPath, read_raw_bids

bids_path = BIDSPath(subject=subject, run=run, task=task,
                     suffix=suffix, datatype=datatype, root=bids_root)
raw = read_raw_bids(bids_path=bids_path, verbose=False)
#print(bids_path)


In [ ]:

# sub052_path, sub052_raw = subject_Accesor('052', '.set')
# print(sub052_raw)

import matplotlib.pyplot as plt
%matplotlib inline
fig=plt.figure(figsize=(15, 15))
raw.plot()

In [ ]:
import pandas as pd
import numpy as np
import mne 
def montagery(n):
    df = pd.read_csv(f'{dataset}/sub-{n}/eeg/sub-{n}_task-Rest_run-{run}_electrodes.tsv', delimiter ='\t').dropna()
    # xyz = electrodes_xyz.to_numpy()
    # chs = electrodes_xyz.to_dict('index')
    # print(xyz)
    # df = pd.read_csv('channels.txt')
    ch_names = df.name.to_list()

    pos = df[['x','y','z']].values
    dig_ch_pos = dict(zip(ch_names,pos))
    # print(dig_ch_pos['CB1'])
    montage = mne.channels.make_dig_montage(ch_pos=dig_ch_pos)
    return montage

# sub_montage = montagery('001')
# sub_montage.plot()

In [ ]:
from mne import io, Epochs
from mne.preprocessing import ICA

#getting raw data
def eeg_epochage(n):
    """Separates raw subject eeg data into epochs, and filters into annotated important events. 
    Returns epochs as Epoch item, and event_id attribute of Epochs item. Needed to not topoplot all 932 epochs, 
    and only plot important events."""
    
    eeglab_raw = mne.io.read_raw_eeglab(f'{dataset}/sub-{n}/eeg/sub-{n}_task-Rest_run-{run}_eeg.set', preload=True)
    
#     print(eeglab_raw.annotations)

#     eeglab_raw.plot_psd(fmax=50)
#     eeglab_raw.plot(duration=5, n_channels=66, show_scrollbars=True)
    
    events_from_annot, event_dict = mne.events_from_annotations(eeglab_raw) #eeglab_raw --> specific to subject '00x'
    # print(event_dict) #likely ea key is the electrode #
    # print(events_from_annot)
    # print(eeglab_raw.ch_names)


    #event_id = dict(aud_l=1, aud_r=2) #event trigger and conditions
    tmin = -0.2  # start of each epoch (200ms before the trigger)
    tmax = 0.5  # end of each epoch (500ms after the trigger)
    #eeglab_raw.set_channel_types({'HEOG': 'eog', 'VEOG': 'eog'})
    picks = mne.pick_types(eeglab_raw.info, meg=False, eeg=True, eog=True, ecg=False, stim=False,
                           exclude='bads')


    baseline = (None, 0)  # means from the first instant to t = 0
    reject = dict(eog=150e-6)
    #event_id=event_dict

    epochs = mne.Epochs(eeglab_raw, events_from_annot, event_dict ,tmin, tmax, proj=True, picks=picks,
                       baseline=baseline, preload=True) #ignored the reject variable, was throwing an error
    epochs.drop_channels('CB1')
    epochs.drop_channels('CB2')
    return epochs, epochs.event_id
    
# sub_epochs, sub_events = eeg_epochage('001')
# print(type(sub_events))

In [ ]:
#Visualizing -- need to make an evks list first
from matplotlib import pyplot as plt
from mne import read_evokeds
# print(epochs.ch_names) 


def visualizer(sub, dep: str):
    """Returns all plots of passed in subjects. Str 'dep' must specify either 
    'Depressed' or 'Control' to save in correct file path name."""
    #, epochs, ids: dict, montage
    epochs, events = eeg_epochage(sub)
    montage = montagery(sub)
    print(events)
    a = 1
    for n in events:
        plt.figure(a, figsize = (7.0,7.0))
        #epochs[n].plot() #this is where the event that was marked; 
        evoked = epochs[n].average() #this is averaging the signals to create an evoked plot  # epoch section 6 is v last; unsure of order of epochs
        #evoked.plot(picks='eeg')
        #topoplot
#         evoked.apply_baseline((None, 0))
        evoked.set_montage(montage, on_missing='ignore')
        fig = plt.gcf()
        mne.viz.plot_topomap(evoked.data[:, 0], evoked.info)
        if int(n)%2 == 0: 
            fig.savefig(f'{dep}_Open/run-{run}/sub-{sub}_event-{n}_topograph{a}.png')
        else:
            fig.savefig(f'{dep}_Closed/run-{run}/sub-{sub}_event-{n}_topograph{a}.png')
        a+=1

# visualizer('001', 'Control')

#from their task.rtf file:
#\cf1 ! 238 (17) is for RSA\cf0\par
# ! 254 (1), 253 (2), 252 (3), 251 (4), 250 (5), 249 (6)  are for frontal asym.\par
# \cf2 ! 244 (11), 243 (12), 242 (13), 241 (14), 240 (15), 239 (16) are for LORETA: 2 second segments\par
# \cf0 ! Odd is closed, Even is open\par

In [ ]:
#Separate subjects based on depressed or not
#load in excel? file -- sort based on BDI range, > or < 10 -- mild to moderate depression = 10-18 inclusive
import re
import glob


participants = pd.read_csv('participants.tsv', delimiter ='\t').set_index('participant_id')
dep_sub = participants.loc[participants['BDI'] >= 10]
control_sub = participants.loc[participants['BDI'] <10]
dep_subject_ids = dep_sub.index.str[4:]
print(dep_subject_ids)

# print('sub-052_task-Rest_run-01_eeg.set' == 'sub-052_task-Rest_run-01_eeg.set')
# print(os.path.exists(f'{dataset}/sub-052/eeg/sub-052_task-Rest_run-01_eeg.set'))

# i = '052'

dep_no_sets = [] #all depressed subjects without .set files

for i in dep_subject_ids:
    find_path = glob.glob(fr'Depressed_.*/sub-{i}_*.png')
    if find_path:  #if the topoplot already exists, then skip making anymore topoplots for that subject
        continue
    else:
        try:
            visualizer(i, 'Depressed')
        except:
            dep_no_sets.append(i)
            pass

print("Could not find .set files for subjects", dep_no_sets)
#need to handle OSError-- if .set file doesn't exist for that subject
print(len(dep_subject_ids) - len(dep_no_sets))

In [ ]:

cont_subject_ids = control_sub.index.str[4:]
print(cont_subject_ids)

# print('sub-052_task-Rest_run-01_eeg.set' == 'sub-052_task-Rest_run-01_eeg.set')
# print(os.path.exists(f'{dataset}/sub-052/eeg/sub-052_task-Rest_run-01_eeg.set'))

# i = '052'

cont_no_sets = [] #all depressed subjects without .set files

for i in cont_subject_ids:
    find_path = glob.glob(fr'Control_.*/sub-{i}_*.png')
    if find_path:  #if the topoplot already exists, then skip making anymore topoplots for that subject
        continue
    else:
        try:
            visualizer(i, 'Control')
        except:
            cont_no_sets.append(i)
            pass

print("Could not find .set files for subjects", cont_no_sets)
#need to handle OSError-- if .set file doesn't exist for that subject
print(len(cont_subject_ids) - len(cont_no_sets))